* 导入数据
* 数据文件与该文件放在同一文件夹

In [ ]:
import numpy as np
import pandas as pd

d_gen=pd.read_csv('GEN-sarc-notsarc.csv')
d_hyp=pd.read_csv('HYP-sarc-notsarc.csv')
d_rq=pd.read_csv('RQ-sarc-notsarc.csv')

设置标签 讽刺为1 非讽刺为0

In [2]:
text1=d_gen["text"]
text2=d_hyp["text"]
text3=d_rq["text"]
lable1=[1]*len(text1)
lable2=[0]*len(text2)
lable3=[0]*len(text3)


合并

In [3]:
texts=[]
labels=[]

texts.extend(text1)
texts.extend(text2)
texts.extend(text3)

labels.extend(lable1)
labels.extend(lable2)
labels.extend(lable3)

* 使用Keras的Tokenizer对句子进行编码

In [5]:
from keras.preprocessing.text import Tokenizer  
# from keras.preprocessing.sequence import pad_sequences  
# 将上面替换为下面
from tensorflow.keras.preprocessing.sequence import pad_sequences
  
# 初始化 Tokenizer 并拟合文本数据  
tokenizer = Tokenizer(num_words=10000)  # 假设我们只想保留最常见的10000个单词  
tokenizer.fit_on_texts(texts)  
  
# 将文本转换为序列  
sequences = tokenizer.texts_to_sequences(texts)  
  
# 填充/截断序列，使其具有相同的长度  
max_len = max([len(seq) for seq in sequences])  
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')  
  

* 使用Keras构建一个基于LSTM的讽刺检测模型

In [7]:
from keras.models import Sequential  
from keras.layers import Embedding, LSTM, Dense  
from sklearn.model_selection import train_test_split  
  
# padded_sequences 是序列，labels 是对应的标签  
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)  
  
# 定义模型  
model = Sequential()  
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len))  
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  
model.add(Dense(1, activation='sigmoid'))   
  
# 编译模型  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  

y_test = np.array(y_test)
y_train = np.array(y_train)

# 训练模型  
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)  
  
# 评估模型  
_, accuracy = model.evaluate(X_test, y_test)  
print(f'Test accuracy: {accuracy:.4f}')

Epoch 1/10
235/235 [==============================] - 288s 1s/step - loss: 0.6184 - accuracy: 0.6950 - val_loss: 0.6202 - val_accuracy: 0.6933
Epoch 2/10
235/235 [==============================] - 269s 1s/step - loss: 0.6158 - accuracy: 0.6950 - val_loss: 0.6165 - val_accuracy: 0.6933
Epoch 3/10
235/235 [==============================] - 253s 1s/step - loss: 0.6164 - accuracy: 0.6950 - val_loss: 0.6168 - val_accuracy: 0.6933
Epoch 4/10
235/235 [==============================] - 244s 1s/step - loss: 0.6163 - accuracy: 0.6950 - val_loss: 0.6171 - val_accuracy: 0.6933
Epoch 5/10
235/235 [==============================] - 252s 1s/step - loss: 0.6159 - accuracy: 0.6950 - val_loss: 0.6165 - val_accuracy: 0.6933
Epoch 6/10
235/235 [==============================] - 255s 1s/step - loss: 0.6161 - accuracy: 0.6950 - val_loss: 0.6169 - val_accuracy: 0.6933
Epoch 7/10
235/235 [==============================] - 251s 1s/step - loss: 0.6159 - accuracy: 0.6950 - val_loss: 0.6165 - val_accuracy: 0.6933

保存模型

In [9]:
model.save('my_model.h5')

* 保存tokenizer

In [11]:
from keras.preprocessing.text import Tokenizer  
import json  
  
# 保存 Tokenizer 的词汇表  
with open('tokenizer.json', 'w', encoding='utf-8') as f:  
    tokenizer_json = tokenizer.to_json()  
    f.write(tokenizer_json)  

# 将 Tokenizer 配置转换为 JSON 字符串  
tokenizer_config = tokenizer.to_json() 

# 将 JSON 字符串保存到文件  
with open('tokenizer_config.json', 'w', encoding='utf-8') as f:  
    f.write(tokenizer_config) 


使用模型进行预测

In [23]:
from keras.models import load_model  
from keras.preprocessing.text import tokenizer_from_json  
import numpy as np  
  
# # 加载保存的 Tokenizer 配置  
# with open('tokenizer.json', 'r', encoding='utf-8') as f:  
#     tokenizer_json = json.load(f)  
#     loaded_tokenizer = tokenizer_from_json(tokenizer_json)  

# 加载整个模型（包括结构和权重）  
model = load_model('my_model.h5')  

# from keras.preprocessing.sequence import pad_sequences  

# new_texts是你想要预测的新文本  
new_texts = ["If that's true, then Freedom of Speech is doomed. Harassment is subjective. Now I can claim that a book I don't like is harassing me, and it will be banned.","Neener neener - is it time to go in from the playground yet?"]  
new_sequences = tokenizer.texts_to_sequences(new_texts)  
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_len)

# 进行预测  
predictions = model.predict(new_padded_sequences)  

# 对于分类问题，如果输出层使用了softmax，你可能需要取概率最高的类别  
predicted_classes = np.argmax(predictions, axis=1)  

# 如果你的标签是整数编码的，你可能需要将这些整数转换回原始的标签名  
label_to_name = {0: '非讽刺', 1: '讽刺'}  # 假设你有一个这样的映射  
predicted_labels = [label_to_name[cls] for cls in predicted_classes]

print(predicted_labels)
# from sklearn.metrics import accuracy_score, classification_report  

# # 假设y_true是真实标签  
# accuracy = accuracy_score(y_true, predicted_classes)  
# report = classification_report(y_true, predicted_classes)  

# print(f'Accuracy: {accuracy:.4f}')  
# print(report)

1/1 [==============================] - 0s 386ms/step
['非讽刺', '非讽刺']
